<a href="https://colab.research.google.com/github/myidispg/TFLite_projects/blob/master/tf_lite_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create a dummy model and fit it to a dummy dataset

In [8]:
import tensorflow as tf

# Fit y = 2x -1 
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]

model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(units=1, input_shape=[1])]
)

model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(x, y, epochs=500)

Train on 6 samples
Epoch 1/500
6/6 [==============================] - 0s 35ms/sample - loss: 18.3888
Epoch 2/500
6/6 [==============================] - 0s 2ms/sample - loss: 14.7256
Epoch 3/500
6/6 [==============================] - 0s 2ms/sample - loss: 11.8383
Epoch 4/500
6/6 [==============================] - 0s 2ms/sample - loss: 9.5614
Epoch 5/500
6/6 [==============================] - 0s 2ms/sample - loss: 7.7651
Epoch 6/500
6/6 [==============================] - 0s 2ms/sample - loss: 6.3468
Epoch 7/500
6/6 [==============================] - 0s 2ms/sample - loss: 5.2260
Epoch 8/500
6/6 [==============================] - 0s 2ms/sample - loss: 4.3395
Epoch 9/500
6/6 [==============================] - 0s 2ms/sample - loss: 3.6374
Epoch 10/500
6/6 [==============================] - 0s 2ms/sample - loss: 3.0804
Epoch 11/500
6/6 [==============================] - 0s 2ms/sample - loss: 2.6377
Epoch 12/500
6/6 [==============================] - 0s 2ms/sample - loss: 2.2850
Epoch 13/500
6

## Save as a tflite model

In [9]:
import pathlib
export_dir = 'saved_model'
tf.saved_model.save(model, export_dir=export_dir)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Save the model
tflite_model_file = pathlib.Path('foo.tflite')
tflite_model_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: saved_model/assets


792

### Try on a pre-trained model

In [10]:
model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
tflite_model_file = pathlib.Path('foo.tflite')
tflite_model_file.write_bytes(tflite_model)

13978384

# Quantize only the weights

In [11]:
conveter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

# Save using Pathlib
tflite_model_file = pathlib.Path('foo_quant_weights.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

3579720

# Quantize the weights + activations

The converted needs kind of a small data as input to determine the ranges etc. and perform "callibration" on the activations and ultimately convert to Integer Quantized model.

In [0]:
def generator():
  data = tfds.load(...)
  for _ in range(num_calibration_steps):
    image, = data.take(1)
    yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimization mode
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Pass the representative datset to the converter
converter.representative_dataset = tf.lite.RepresentativeDataset(generator)

# Some operations might not have quantized implementation. They will be in original float values.
converter.target_spec.supported_ops = [t.lite.OpsSet.TFLITE_BUILTINS_INT8]